# Open-Meteo : Weather Forecast API

Open-Meteo features weather forecasts aswell as historical weather data, with up to 1km resolution.

We'll use the free available licence for non-commercial use, which restricts its usage to less than 10 000 daily requests.

Forecast documentation : https://open-meteo.com/en/docs

Historical Data Documentation : https://open-meteo.com/en/docs/historical-weather-api

# Imports

In [24]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

from geopy.geocoders import Nominatim
import numpy as np

# Documentation

Weather Variables : WMO Weather interpretation codes (WW)

In [25]:
wmo_codes = pd.DataFrame({"code": [0, 1, 2, 3, 45, 48, 51, 53, 55, 56, 57, 61, 63, 65, 66, 67, 71, 73, 75, 77, 80, 81, 82, 85, 86, 95, 96, 99]})
wmo_codes['description'] = np.select(
    [wmo_codes['code'].isin([0]),
     wmo_codes['code'].isin([1, 2, 3]),
     wmo_codes['code'].isin([45, 48]),
     wmo_codes['code'].isin([51, 53, 55]),
     wmo_codes['code'].isin([56, 57]),
     wmo_codes['code'].isin([61, 63, 65]),
     wmo_codes['code'].isin([66, 67]),
     wmo_codes['code'].isin([71, 73, 75]),
     wmo_codes['code'].isin([77]),
     wmo_codes['code'].isin([80, 81, 82]),
     wmo_codes['code'].isin([85, 86]),
     wmo_codes['code'].isin([95]),
     wmo_codes['code'].isin([96, 99]),
    ],
    ['Clear sky',
     'Mainly clear, partly cloudy, and overcast',
     'Fog and depositing rime fog',
     'Drizzle: Light, moderate, and dense intensity',
     'Freezing Drizzle: Light and dense intensity',
     'Rain: Slight, moderate and heavy intensity',
     'Freezing Rain: Light and heavy intensity',
     'Snow fall: Slight, moderate, and heavy intensity',
     'Snow grains',
     'Rain showers: Slight, moderate, and violent',
     'Snow showers slight and heavy',
     'Thunderstorm: Slight or moderate',
     'Thunderstorm with slight and heavy hail'
    ]
)
print("WMO weather codes :")
display(wmo_codes)

WMO weather codes :


,code,description
0,0,Clear sky
1,1,"Mainly clear, partly cloudy, and overcast"
2,2,"Mainly clear, partly cloudy, and overcast"
3,3,"Mainly clear, partly cloudy, and overcast"
4,45,Fog and depositing rime fog
5,48,Fog and depositing rime fog
6,51,"Drizzle: Light, moderate, and dense intensity"
7,53,"Drizzle: Light, moderate, and dense intensity"
8,55,"Drizzle: Light, moderate, and dense intensity"
9,56,Freezing Drizzle: Light and dense intensity


# Simple requests

## Documentation's example usecase

Getting the weather forecasts at ESME's Ivry-Sur-Seine campus.

In [26]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after=3600)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": 48.81452162820077,
	"longitude": 2.3948078406657114,
	"hourly": ["temperature_2m", 
            "relative_humidity_2m", 
            "wind_speed_10m",
            "wind_direction_10m",
            "wind_speed_80m",
            "wind_direction_80m",
            "apparent_temperature",
            "pressure_msl",
            "cloud_cover",
            "precipitation", 
            "weather_code",
            "is_day"
            ],
    "timezone": "auto"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]

Quick check of the selected address

In [27]:
geolocator = Nominatim(user_agent="ESMEAirPollutionPrediction")
location = geolocator.reverse(f"{params['latitude']}, {params['longitude']}")
print(location.address)

60, Boulevard de Brandebourg, Ivry Port, Ivry-sur-Seine, L'Haÿ-les-Roses, Val-de-Marne, Île-de-France, France métropolitaine, 94200, France


In [28]:
print(f"Coordinates {response.Latitude()}°E {response.Longitude()}°N")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()

hourly_data = {
    "date": pd.date_range(
        start=pd.to_datetime(hourly.Time(), unit = "s"),
        end=pd.to_datetime(hourly.TimeEnd(), unit = "s"),
        freq=pd.Timedelta(seconds=hourly.Interval()),
        inclusive="left"
        )
    }
for i in range(hourly.VariablesLength()):  # Iterating through all the variables in params["hourly"]
    hourly_data[params["hourly"][i]] = hourly.Variables(i).ValuesAsNumpy()

hourly_dataframe = pd.DataFrame(data=hourly_data)
display(hourly_dataframe)

Coordinates 48.81999969482422°E 2.3999996185302734°N
Elevation 36.0 m asl
Timezone b'Europe/Paris' b'CET'
Timezone difference to GMT+0 3600 s


,date,temperature_2m,relative_humidity_2m,wind_speed_10m,wind_direction_10m,wind_speed_80m,wind_direction_80m,apparent_temperature,pressure_msl,cloud_cover,precipitation,weather_code,is_day
0,2023-11-23 23:00:00,11.2065,86.0,11.525623,284.470337,23.143827,285.333344,9.251688,1023.700012,100.0,0.0,3.0,0.0
1,2023-11-24 00:00:00,11.2565,87.0,11.720751,280.619598,23.749088,284.036255,9.332809,1023.299988,98.0,0.0,3.0,0.0
2,2023-11-24 01:00:00,11.2565,88.0,10.948973,279.462250,21.915693,284.264557,9.491747,1021.599976,100.0,0.0,3.0,0.0
3,2023-11-24 02:00:00,11.3065,89.0,10.086427,272.045349,20.364674,278.130005,9.727613,1021.500000,100.0,0.0,3.0,0.0
4,2023-11-24 03:00:00,11.1565,90.0,9.605998,282.994629,20.620806,286.220276,9.652798,1020.599976,50.0,0.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,2023-11-30 18:00:00,5.9995,71.0,22.530937,257.074005,45.891174,258.690094,0.771378,997.700012,74.0,0.0,2.0,0.0
164,2023-11-30 19:00:00,5.3495,72.0,21.046917,257.152283,42.572948,258.785126,0.267219,998.599976,64.0,0.0,1.0,0.0
165,2023-11-30 20:00:00,4.4995,72.0,19.039200,254.649811,39.125893,257.242462,-0.418739,999.500000,53.0,0.0,1.0,0.0
166,2023-11-30 21:00:00,3.7995,74.0,17.309973,253.072403,35.538033,256.526825,-0.911877,1000.200012,43.0,0.0,1.0,0.0


In [29]:
hourly_dataframe.to_csv(f"../data/forecast_{str(hourly_dataframe.at[0, 'date']).split()[0]}_hourly.csv")  # saved to check later and compare it to real life values

## Minutely-15

In [30]:
params_minutely = {
	"latitude": 48.81452162820077,
	"longitude": 2.3948078406657114,
    "minutely_15": ["temperature_2m", 
            "relative_humidity_2m", 
            "wind_speed_10m",
            "wind_direction_10m",
            "wind_speed_80m",
            "wind_direction_80m",
            "apparent_temperature",
            # "cloud_cover",
            "precipitation", 
            "weather_code",
            # "is_day"
            ],
    "timezone": "auto"
}
responses = openmeteo.weather_api(url, params=params_minutely)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]

In [31]:
minutely = response.Minutely15()
minutely_data = {
    "date": pd.date_range(
        start=pd.to_datetime(minutely.Time(), unit = "s"),
        end=pd.to_datetime(minutely.TimeEnd(), unit = "s"),
        freq=pd.Timedelta(seconds=minutely.Interval()),
        inclusive="left"
        )
    }
for i in range(minutely.VariablesLength()):  # Iterating through all the variables in params["hourly"]
    minutely_data[params_minutely["minutely_15"][i]] = minutely.Variables(i).ValuesAsNumpy()

minutely_dataframe = pd.DataFrame(data=minutely_data)
display(minutely_dataframe)

,date,temperature_2m,relative_humidity_2m,wind_speed_10m,wind_direction_10m,wind_speed_80m,wind_direction_80m,apparent_temperature,precipitation,weather_code
0,2023-11-23 23:00:00,11.20,86.0,11.525623,284.470337,23.143827,285.333344,9.243475,0.0,3.0
1,2023-11-23 23:15:00,11.20,86.0,11.525623,284.470337,23.491207,285.101135,9.243475,0.0,3.0
2,2023-11-23 23:30:00,11.20,87.0,11.792404,282.339081,23.491207,285.101135,9.250783,0.0,3.0
3,2023-11-23 23:45:00,11.25,87.0,11.792404,282.339081,23.749088,284.036255,9.314123,0.0,3.0
4,2023-11-24 00:00:00,11.25,87.0,11.720751,280.619598,23.749088,284.036255,9.324573,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...
283,2023-11-26 21:45:00,3.50,84.0,9.178235,191.309891,21.868332,200.224945,0.204375,0.0,3.0
284,2023-11-26 22:00:00,3.50,84.0,9.255571,193.495743,21.659916,201.447800,0.193096,0.0,3.0
285,2023-11-26 22:15:00,3.50,84.0,9.255571,193.495743,21.659916,201.447800,0.193096,0.0,3.0
286,2023-11-26 22:30:00,3.50,84.0,9.605998,192.994614,21.868332,200.224945,0.141993,0.0,3.0


In [32]:
minutely_dataframe.to_csv(f"../data/forecast_{str(hourly_dataframe.at[0, 'date']).split()[0]}_minutely.csv")  # saved to check later and compare it to real life values

Utiliser directions et vitesses de vents

comparer forecasts au réel

voir meteofrance directement ?
